# Exploring LittlePay's Authorisations Data

In [2]:
from calitp import get_engine, query_sql, magics

In [3]:
import os
os.environ['AIRFLOW_ENV'] = 'development'

## Initial exploration

* How big is the table?
* What are the fields available?

In [6]:
%%sql

select count(*) from payments.stg_enriched_authorisations

,f0_
0,55949


In [7]:
%%sql

select *
from payments.stg_enriched_authorisations
limit 10

,participant_id,aggregation_id,acquirer_id,request_type,transaction_amount,currency_code,retrieval_reference_number,response_code,status,authorisation_date_time_utc,calitp_extracted_at,calitp_hash,calitp_file_name,calitp_export_account,calitp_export_datetime,calitp_n_dupes,calitp_n_dupe_ids,calitp_dupe_number
0,sbmtd,e56cdf95-d371-403a-9f74-306a6fe90a71,CS,CARD_CHECK,0.0,840,None,None,UNAVAILABLE,2021-09-29 01:46:38+00:00,2022-02-22,Wh69ea2YsKVmvq6sg9oXZw==,gs://gtfs-data-test/payments-processed/authori...,sbmtd,2021-09-30 04:44:00,1,1,1
1,mst,3854286c-19b3-48a3-9d30-60e28bdc13de,CS,CARD_CHECK,0.0,840,6207816454776574504053,100,VERIFIED,2021-05-12 01:07:26+00:00,2022-02-22,W9Qp2/mBFdAKtg2kH6Plcw==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,2
2,mst,3854286c-19b3-48a3-9d30-60e28bdc13de,CS,CARD_CHECK,0.0,840,6207816454776574504053,100,VERIFIED,2021-05-12 01:07:26+00:00,2022-02-22,W9Qp2/mBFdAKtg2kH6Plcw==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,1
3,mst,9335c6ca-fd5a-41ac-9b35-c6d97eb2b14c,CS,CARD_CHECK,0.0,840,6207937648856229104091,100,VERIFIED,2021-05-12 04:29:25+00:00,2022-02-22,S6QFtaJkO692FXM8Vjj2kw==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,2
4,mst,9335c6ca-fd5a-41ac-9b35-c6d97eb2b14c,CS,CARD_CHECK,0.0,840,6207937648856229104091,100,VERIFIED,2021-05-12 04:29:25+00:00,2022-02-22,S6QFtaJkO692FXM8Vjj2kw==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,1
5,mst,4a9e5e03-8174-46f7-a727-2bc15c79375b,CS,CARD_CHECK,0.0,840,6208373628036277803280,100,VERIFIED,2021-05-12 16:36:03+00:00,2022-02-22,LPTi4yxv7iA3FQIlVLmYvA==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,1
6,mst,4a9e5e03-8174-46f7-a727-2bc15c79375b,CS,CARD_CHECK,0.0,840,6208373628036277803280,100,VERIFIED,2021-05-12 16:36:03+00:00,2022-02-22,LPTi4yxv7iA3FQIlVLmYvA==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,2
7,mst,8c4b2b3c-1137-4410-9d95-363cb7e03c76,CS,CARD_CHECK,0.0,840,6208396169716260904066,100,VERIFIED,2021-05-12 17:13:37+00:00,2022-02-22,OIw44OdpQI55TQkksdzYpg==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,1
8,mst,8c4b2b3c-1137-4410-9d95-363cb7e03c76,CS,CARD_CHECK,0.0,840,6208396169716260904066,100,VERIFIED,2021-05-12 17:13:37+00:00,2022-02-22,OIw44OdpQI55TQkksdzYpg==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,2,2,2
9,mst,9f133f34-be3a-4864-bbf2-1ee9d788211d,CS,CARD_CHECK,0.0,840,6208414965046636204080,100,VERIFIED,2021-05-12 17:44:57+00:00,2022-02-22,35FbjXzx+CamNhL+tHrF3A==,gs://gtfs-data-test/payments-processed/authori...,mst,2021-06-18 02:24:00,1,1,1


We probably want to join with the `micropayments` table on the `aggregation_id`. According to the LittlePay API docs document:

> The aggregation that is being authorised. A single aggregation can have multiple
> authorisations. The authorisations that are submitted depend on the scheme and the
> result of the initial authorisation (if declined, debt recovery authorisations may
> be performed).

It looks like there are duplicates (even just in the table above), so we'll have to deduplicate on the hash. We may also want to ensure that the `aggregation_id` values are sensible with respect to the micropayments table.

In [9]:
%%sql

# Number of unique records
select count(*)
from payments.stg_enriched_authorisations
where calitp_dupe_number = 1

,f0_
0,38490


In [18]:
%%sql

select micropayments.*
from payments.stg_enriched_authorisations auths
full outer join payments.stg_cleaned_micropayments micropayments using (aggregation_id)
where auths.aggregation_id is null
# where (micropayments.aggregation_id is null and auths.calitp_dupe_number = 1)
limit 10

,micropayment_id,aggregation_id,participant_id,customer_id,funding_source_vault_id,transaction_time,payment_liability,charge_amount,nominal_amount,currency_code,type,charge_type,calitp_extracted_at,calitp_hash,calitp_export_account,calitp_export_datetime
0,5e9f9edb-3737-4783-a776-f3fdfb64969c,e210fdb4-98dc-4299-8310-14ea8b228ad7,mst,ff8b47a6-635b-41c8-894f-e6d36e49595f,32cce0ef-7bae-4aae-9024-8d24b2332ebf,2021-11-02 15:34:32+00:00,OPERATOR,1.50,1.50,840,DEBIT,complete_variable_fare,2022-02-22,iVCRMgVJP/AgR8/mSx0yPw==,mst,2021-11-04 05:00:00
1,a036859b-bbbe-4199-a4bd-a5af3f3f75c4,42891f67-73cf-416b-bcf7-4cb62693e542,mst,a3e05871-e372-48a2-a6bb-9d2fdc7a5eff,4363627a-a172-482e-bd64-b319dd819538,2021-11-17 16:29:16+00:00,OPERATOR,1.50,1.50,840,DEBIT,complete_variable_fare,2022-02-22,XQHM3t6MyOx2qF0W2fYRQA==,mst,2021-11-19 05:07:00
2,ca9c6918-6288-42e6-8905-bfeeeb4cc45b,b8e1f34f-0ec4-4b03-8ed1-221a09dd138b,mst,954de086-5d20-4efd-8191-4eb9cfa76e29,ab767bd2-dab5-40b8-9211-1749f3e15456,2021-07-26 19:58:38+00:00,OPERATOR,1.50,1.50,840,DEBIT,complete_variable_fare,2022-02-22,sz9/rnjO5zrFl7pIMwYawQ==,mst,2021-07-28 05:28:00
3,3b121071-911a-493d-bf02-312710ffba78,c36d25ac-86c5-4432-bd15-45b603a0bcdf,sbmtd,1bd04497-57e5-4391-b125-f6b23a682117,0197e6a5-1254-4cce-bfa4-4d15f8480677,2021-08-31 15:45:50+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,NaV0OydyXSw3g/VqkraRiw==,sbmtd,2021-10-18 23:16:00
4,5a8dcb07-4c9e-404c-aaa9-694cb4fcfb5f,3c716d23-fd36-44e1-9221-d8e0404cefb0,sbmtd,adc2bd46-6ecd-4f5a-a1b9-f01005cb7804,1046a428-8d96-4e18-99c8-78740f9c105c,2021-08-15 21:00:00+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,lGaAYfNT0gzACZSx8UhHXw==,sbmtd,2021-10-18 23:16:00
5,7670b845-eb45-49db-9497-8c34c21de6a2,5417499d-34f1-48fe-820e-1ccc722b52e1,sbmtd,86e09e30-9be6-4499-b93d-98d78b9e0c11,2fa8e74d-e8a3-4910-ae90-9529463cf272,2021-08-13 20:35:41+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,now4nmQTHT9lzra7HEm5NA==,sbmtd,2021-10-18 23:16:00
6,979d5c9b-ef6b-4e02-a39b-810e5d3907b1,ef7b7afb-a4ec-4826-bec0-f0c69e7215e1,sbmtd,06bcf84f-e555-41ce-a52a-317dbad245ca,42faa21a-1e70-4374-9d6e-7b26aa661202,2021-09-06 04:20:34+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,YX7u2Y5wbeAYWH6hk7Xc0Q==,sbmtd,2021-10-18 23:16:00
7,16da17c2-5a27-4950-8e23-12599bca1243,06467d45-9225-409f-a1a5-0e4dcf131e47,sbmtd,6c1e9107-8ce3-4d3a-9757-694a573d07f4,4fa78704-639b-43b8-b086-25ac606e6f1f,2021-08-11 22:26:21+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,W3iO33JzXF7UKbTCCfKONA==,sbmtd,2021-10-18 23:16:00
8,362e56b5-93b7-489f-8065-21938578d0de,baa12bf4-6014-489b-960c-b5c2ff35ca2d,sbmtd,dba6f50b-38a7-4d88-bad3-349848bcedd0,533ad61d-1a43-4810-8acb-aa1b0d3d4287,2021-09-10 05:42:47+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,tTb/j30ipU5JwNx/yz2TuQ==,sbmtd,2021-10-18 23:16:00
9,3a958410-c167-4b8b-9a69-0902070dabdf,3b806213-d105-4905-93ce-aa6fb5dae2de,sbmtd,59fcf16e-408e-410b-83c7-3c0452dede56,7fb00598-8851-467c-8d9e-7a4bcdd6a89d,2021-08-30 14:26:18+00:00,OPERATOR,1.75,1.75,840,DEBIT,flat_fare,2022-02-22,WEmXx2zZagI/sWaOIKZOlw==,sbmtd,2021-10-18 23:16:00
